# Convolutional Neural Network

### Importing the libraries

In [2]:
# tensorflow ve görüntü ön işleme adımlarını gerçekleştirmek için keras kütüphanesinden ImageDataGenerator ekledik
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

In [3]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

*Resimlerde ön işleme adımının gerçekleşme sebebi overfitting dediğimiz aşırı çoğalma olayını engellemektir.*

*Data Preprocessing bölümünde train ve test görüntüleri üzerinde bir ön işleme adımı gerçekleştiriyor olacağız.Bu işlemlerden bahsetmek gerekirse;*

*   Eğitim için kullanacağımız görüntülerimizi yakınlaştırma(zoom) ,yatay eksende döndürme(horizontal rotation) , resimler üzerinde ölçeklendirme(rescale) işlemlerini Keras ile birlikte ImageDataGenerator classından yapıyor olacağız.
*   Test görüntülerini eğitim de kullanacağımız train görüntüleri gibi yakınlaştırma veya döndürme işlemleri yapmıyoruz yani bunları istemiyoruz çünkü test etmek için orjinal kalmasını istiyor olacağız fakat ölçeklendirme işlemi gerçekleştiriliyor olacaktır.

### Preprocessing the Training set

In [4]:
# Eğitim veri seti için ön işleme nesnesi oluşturma , oluşturduğumuz train_datagen değişkeni tüm resimlerde dönüşüm işlemi uygulamamızı sağlayan nesne olacaktır.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# Eğitim veri setlerine dosya yolu ile ulaşma ve preprocessing nesnesini içersinde kullanmamızı sağlayan fonksiyon
training_set = train_datagen.flow_from_directory(
        '/home/zeyneloglu/Desktop/DeepLearning/CNN/dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


  *Yukarıda gerçekleştirdiğimiz işlemleri kısaca özetlemek gerekirse train_datagen değişkeni aslında ImageDataGeneretor sayesinde resimler üzerinde ölçeklendirme , zoom_range(yakınlaştırma ayarı) , horizontal_flip(yatay dönüş) özelliklerini yapmasını sağlar ve daha birçok özellik vardır ayrıntısına girip bakabiliriz.*

  *training_set değişkeni ise hedef resimlerimi seçip onun üzerinden train_datagen nesnesimi uygulamamızı ve aynı şekilde target_size(hedef eğitilecek resim boyutu) , batch_size(model eğitildikten sonra gösterilecek resim sayısı) , class_mode(çıktının türü)--> biz köpek ve kedi mi öğrenmesini istediğimiz için binary diyebiliriz.*

### Preprocessing the Test set

In [5]:
# Test resimleri üzerinde preprocessing işlemi uygulama [train setinde farklı olarak özelliklerin sadece ölçeklendirmesini kullandık çünkü orjinal bir şekilde kalmasını istiyoruz]
test_datagen = ImageDataGenerator(rescale=1./255)

# Train setinde yaptığımız gibi aynı işlemleri test setleri içinde uyguluyoruz.
test_set= test_datagen.flow_from_directory(
        '/home/zeyneloglu/Desktop/DeepLearning/CNN/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
# CNN nesnesini oluşturduğumuz kısımdır ve oluşturduğumuz nesne ile birlikte diğer adımları gerçekleştiriyor olacağız.
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32 ,kernel_size=3 , activation='relu',input_shape = [64, 64 ,3]))

*Bu katman CNN’nin ana yapı taşıdır. Resmin özelliklerini
algılamaktan sorumludur. Bu katman, görüntüdeki düşük ve yüksek
seviyeli özellikleri çıkarmak için resme bazı ﬁtreler uygular. Örneğin,
bu ﬁltre kenarları algılayacak bir ﬁltre olabilir. Bu ﬁltreler genellikle
çok boyutludur ve piksel değerleri içerirler.(5x5x3) 5 matrisin
yükseklik ve genişliğini, 3 matrisin derinliğini temsil eder.*

***cnn.add(tf.keras.layers.Conv2D())***: *Yapısı ile evrişim katmanını ekliyor olacağız.Conv2D tensorflow 2 convutional işlemi yani evrişim işlemi gerçekleştirecek olan fonksiyondur.Bunun ile ilgili bilmemiz gereken 3 önemli parametre vardır;*

*   filters : Kaç adet filtere haritası kullanılacak ise sayısı
*   kernel_size : Kullanılacak olan filtelerin kaça kaçlık olduğunun sayısı 
*   activation : aktivasyon fonksiyonunu belirttiğimiz parametredir ve genellikle 'relu' kullanılır.
*   input_shape : Görüntünün kaça kaçlık olduğunu ve derinliği [Bunu preprocessing bölümünde target size de yazdığımız sayıdır  , derinliği ise resimler RGB olduğunu için 3'dür diyebiliriz.**Sadece ilk eklendiğinde girilir tekrar evrişim katmanı eklemek istediğimizde bunu parametre olarak eklemeyiz.**

### Step 2 - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 , strides=2))

*Bu katman, CovNet’teki ardışık convolutional katmanları arasına
sıklıkla eklenen bir katmandır. Bu katmanın görevi, gösterimin
kayma boyutunu ve ağ içindeki parametreleri ve hesaplama sayısını
azaltmak içindir. Bu sayede ağdaki uyumsuzluk kontrol edilmiş olur.
Birçok Pooling işlemleri vardır, fakat en popüleri max pooling’dir.
Yine aynı prensipte çalışan average pooling, ve L2-norm pooling
algoritmalarıda vardır.Burada MaxPool işlemini uygalayacağız.*

***cnn.add(tf.keras.layers.MaxPool2D())*** : *Yapısı ile pooling(havuzlama) katmanını ekleyebilriz.Önemli iki parametresi bulunmaktadır;*


*   pool_size : pool yapacak matrixin kaça kaçlık boyuta sahip olduğunu yazarız
*   strides : input üzerinde gezinirken kaç adım kaymasını istediğimiz sayıdır.

### Adding a second convolutional layer

In [10]:
# input_shape parametresini eklemiyoruz çünkü bu bir input girişi değil tekrar bir convolutional katmanı ekliyoruz
cnn.add(tf.keras.layers.Conv2D(filters=32 ,kernel_size=3, activation='relu'))

### Adding a second pooling layer

In [11]:
# İkinci bir Evrişim (convolutional) işleminden sonra tekrar pooling işlemi gerçekleştiriyoruz.
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 , strides=2))

### Step 3 - Flattening

In [12]:
# Yapay sinir ağı oluşturmak için son adım olarak işlemlerin sonucunu tek boyutlu matrix yani vektör haline getirir.
cnn.add(tf.keras.layers.Flatten())

*Bu katmanın görevi basitçe, son ve en önemli katman olan Fully
Connected Layer’ın girişindeki verileri hazırlamaktır. Genel olarak,
sinir ağları, giriş verilerini tek boyutlu bir diziden alır. Bu sinir
ağındaki veriler ise Convolutional ve Pooling katmanından gelen
matrixlerin tek boyutlu diziye çevrilmiş halidir.*

**cnn.add(tf.keras.layers.Flatten())** : *Yapısı ile gerçekleştirilir ve parametre almaz.*

### Step 4 - Full Connection

*Bu katman ConvNet’in son ve en önemli katmanıdır. Verileri
Flattening işleminden alır ve Sinir ağı yoluyla öğrenme işlemini
geçekleştirir.*

*ANN yapısında oluşturduğumuz gibi hidden layer katmanlarını oluşturuyoruz resim üzerinde gerçekleştirdiğimiz adımları Flattening adımı ile vektör haline getirdik ve artık sanki bir yapay sinir ağı eğitiyormuş gibi davranabiliyoruz.


*   units : nöron sayısı
*   activation : aktivasyon fonksiyonu



In [13]:
# hidden layer
cnn.add(tf.keras.layers.Dense(units=128 , activation='relu'))

### Step 5 - Output Layer

In [14]:
#  output layer [binary bir sonuç alacağımız için ya köpek diyecek ya da kedi bu yüzden activation fonksiyonu olarak sigmoid kullanmak daha doğru olacaktır output layer için]
cnn.add(tf.keras.layers.Dense(units=1 , activation='sigmoid'))

## Part 3 - Training the CNN

Bu bölümde oluşturduğumuz yapay sinir ağı modelini veri setimiz için eğitme işlemi gerçekleştiriyor olacağız.

### Compiling the CNN

CNN derleme işlemi gerçekleştiriyor olacağız.Bunun için **cnn.compile()** fonksiyonunu kullanıyor olacağız.

**optimizer:** optime eden fonksiyondur ve en iyilerinden biri **adam** fonksiyonudur.

**loss:**: kayıp fonksiyonu şuan biz veri setimizde binary bir sonuç yani (ikili bir sonuc 0 ve 1 den oluşan) değer alacağımız için 
**binary_crossentropy** kullanmamız uygun olacaktır fakat kategorik bir output alacak isek **category_crossentropy** kullanmamız gerecektir.

**metrics**: eğitim sırasında görmek istediğimiz metrics türlerini bir liste içerisinde yazabilir şuan için accuracy(yani doğruluk skorunu) yazdık fakat araştırarak diğer metrics türlerinide yazabiliriz.

In [15]:
cnn.compile(optimizer='adam', loss ='binary_crossentropy',metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

Veri setinin artık eğitme kısmına geldik bunun için eğitim setleri yukarıda oluşturduğumuz traning_set olacaktır ve test setleri ise parametre olarak validation_set parametresinde yer alır, makine öğrenmesinde olduğu gibi burada da fit etme işlemini kullanıyor olacağız.

**ann.fit(x = traning_set , validation_data = test_set , batch_size , epochs)** parametlerini almaktadır.

**batch_size**: kendi deneylerimiz sonucu optimum değeri bulabiliriz bunun ayrıntılarına internet üzerinde araştırabiliriz eğer hiçbir şey bilmiyorsak 32 uygun bir değerdir.

**epochs**: eğitim adım sayısı gibi düşünebilirz çok fazla olması overfitting sebep olabilir bu yüzden optimum bir sonuç yazmak önemlidir bu veri seti için 100 değeri gireceğiz.

In [16]:
# epochs sayısı seçimi deneme yanılma ile ortaya çıkartılması gerekmektedir.Fakat biz genel bir sayı verdik.
cnn.fit(x = training_set , validation_data = test_set , epochs = 25)

Epoch 1/25
250/250 [==============================] - 22s 87ms/step - loss: 0.6915 - accuracy: 0.5523 - val_loss: 0.6504 - val_accuracy: 0.6460
Epoch 2/25
250/250 [==============================] - 21s 86ms/step - loss: 0.6374 - accuracy: 0.6428 - val_loss: 0.6123 - val_accuracy: 0.6950
Epoch 3/25
250/250 [==============================] - 23s 92ms/step - loss: 0.5795 - accuracy: 0.6991 - val_loss: 0.5387 - val_accuracy: 0.7480
Epoch 4/25
250/250 [==============================] - 23s 90ms/step - loss: 0.5322 - accuracy: 0.7324 - val_loss: 0.5088 - val_accuracy: 0.7470
Epoch 5/25
250/250 [==============================] - 23s 90ms/step - loss: 0.5126 - accuracy: 0.7502 - val_loss: 0.4946 - val_accuracy: 0.7600
Epoch 6/25
250/250 [==============================] - 23s 93ms/step - loss: 0.4829 - accuracy: 0.7681 - val_loss: 0.5027 - val_accuracy: 0.7595
Epoch 7/25
250/250 [==============================] - 22s 90ms/step - loss: 0.4789 - accuracy: 0.7715 - val_loss: 0.4828 - val_accuracy:

## Part 4 - Making a single prediction

*Bu bölümde tek bir resim yükleyerek resmin kedi mi yoksa köpek mi olduğunu tahmin etme işlemi gerçekleştiriyor olacağız.Bu işlemi gerçekleştirmek için birkaç adım izlememiz gerekmektedir;*


*   *İlk olarak resimi yükleme işlemi gerçekleştiriyor olacağız ve bu işlemi keras kütüphanesi içerisinden preprocessing ile birlikte image sınıfından* **load_image(resmin_url , target_size)** *fonksiyonu ile gerçekleştiriyor olacağız.*
*   *Resimi ekledikten sonra bu resmi vektör haline yani dizi haline getirmemiz gerekmektedir çünkü biz biliyoruzki resimler flatting bölümünde tek boyutulu matrixler haline gelmektedir.*
*   *Dizi haline getirdikten sonra numpy kütüphanesi sayesinde tahmin yapacağımız resmimizi başta giriş resimlerinin boyutu (target_size) belirlediysek o boyutta olması için* **np.expand_dims(test_image, axis = 0)** *şeklinde gösteriyor olacağız.*

*   **result = cnn.predict(test_image)** *fonksiyonu ile bu değeri bir değişkene eşitleme işlemi yapıyor olacağız.*
*   *Tahmin değerlerine tam ulaşmak için* **training_set.class_indices**
*   if result[0][0] == 1: --> ilk [0] tahmin değerlerine girer ve ikinci [0] dog [1] kediyi temsil etmektedir.
 *      prediction = 'dog'
*    else:
 *     prediction = 'cat' 




In [44]:
# Gerekli kütüphaneler
import numpy as np
from keras.preprocessing import image

# Tahmin edilecek resmi yükleme
test_image = image.load_img('/home/zeyneloglu/Desktop/DeepLearning/CNN/dataset/single_prediction/cat_or_dog_8.jpg', target_size = (64, 64))
# Diziye çevirme
test_image = image.img_to_array(test_image)
# Target size uyumunu gerçekleştirme
test_image = np.expand_dims(test_image, axis = 0)
# Tahmin sonucunu değişkene aktarma
result = cnn.predict(test_image)
training_set.class_indices
# Tahmin değerini alma
if result[0][0] > 0.5:
  prediction = 'dog'  
else:
  prediction = 'cat'

In [45]:
print(prediction)

cat
